**Table of contents**<a id='toc0_'></a>

-   1. [Using Sympy](#toc1_)
    -   1.1. [Propositional Logic](#toc1_1_)
    -   1.2. [Predicate Logic](#toc1_2_)
-   2. [Using z3 solver](#toc2_)

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


In [2]:
import sympy as sp
from sympy import symbols
from sympy.logic.boolalg import And, Implies, Not
from sympy.logic import inference

sp.__version__

'1.13.3'

# 1. <a id='toc1_'></a>[Using Sympy](#toc0_)


-   Major premise: All humans are mortal.
-   Minor premise: All Greeks are humans.
-   Conclusion/Consequent: All Greeks are mortal.


In [3]:
from utils.logical_syllogism import PropositionalLogicSyllogism
from utils.logical_syllogism import print_satisfaible_worlds

In [4]:
# propositions
propositions = [
    "G & ~Ev",
    "~Ev",  # Evil exists.
    "~G >> R",
    "~C & D >> G",
    "G >> ~R",
]

conclusion = "R"  # Testing if this leads to an inconsistency (contradiction).

## 1.1. <a id='toc1_1_'></a>[Propositional Logic](#toc0_)


In [5]:
PropositionalLogicSyllogism.check_entailment(
    premises=propositions,
    conclusion=conclusion,
)

['G & ~Ev', '~Ev', '~G >> R', '~C & D >> G', 'G >> ~R'] ⊨ R is:


False

In [6]:
problem = PropositionalLogicSyllogism(propositions)
display(problem.representation(simplify=True))
valid_worlds: list[dict] = problem.solve(negate=False)
print_satisfaible_worlds(valid_worlds)
sp.pprint(valid_worlds)  # valid_worlds

'G \\wedge \\neg C \\wedge \\neg Ev \\wedge \\neg R'

Valid in 1 world: satifiable
-------logic is satisfiable in some possible worlds--------
    C         Ev        G         R     
 ------------------------------------
    0         0         1         0     
[{C: False, Ev: False, G: True, R: False}]


## 1.2. <a id='toc1_2_'></a>[Predicate Logic](#toc0_)


In [7]:
from sympy.logic.boolalg import Implies
from sympy import symbols, Symbol
from sympy.logic.inference import satisfiable


# Define the Predicate class
class Predicate:
    def __init__(self, name):
        self.name = name

    def __call__(self, *args):
        args_str = ",".join(map(str, args))
        return Symbol(f"{self.name}({args_str})")


# Define our predicates
M = Predicate("M")  # Predicate P(x)
H = Predicate("H")  # Predicate H(y)
G = Predicate("G")  # Predicate B(x, y)

# Define variables
x1 = Symbol("x1")
# y1 = Symbol("y1")

# Build the formula: ∀x. P(x) → (∃y. H(y) ∧ B(x, y))
# We'll demonstrate this for specific instances x1 and y1
formula = Implies(P(x1), H(y1) & B(x1, y1))

# Print the formula
print("Logical Formula:")
print(formula)

# Check if the formula is satisfiable
results = satisfiable(formula, all_models=True)
print("\nIs the formula satisfiable?")
for r in results:
    print(r is not False)

    if r:
        print("\nOne possible satisfying assignment:")
        for key, value in r.items():
            print(f"{key}: {value}")


NameError: name 'P' is not defined

# 2. <a id='toc2_'></a>[Using z3 solver](#toc0_)


In [ ]:
from z3 import Bools, Solver, And, Bool
problem.symbols
s = Solver()
sym_mapto_z3 = {str(x):Bool(str(x)) for x in problem.symbols}
s.add(sym_mapto_z3['G'])
s.add(sym_mapto_z3['R'] | sym_mapto_z3['G'])
s

[G, Or(R, G)]

In [ ]:
def propositional_logic_syllogism_z3():
    from z3 import Bool, And, Implies, Not, Solver, unsat, prove

    # Define atomic propositions
    P = Bool("P")  # P: All humans are mortal.
    Q = Bool("Q")  # Q: All Greeks are humans.
    R = Bool("R") 
    R = "P & Q"
    print(R)
    R = Bool("R")  # R: All Greeks are mortal.
    # Create a solver
    s = Solver()

    # Add premises (assume P and Q are true)
    s.add(P)
    s.add(Q)

    # Attempt to prove that P and Q imply R
    # In propositional logic, without additional implications, we cannot derive R from P and Q
    # Let's check if (P ∧ Q ∧ ¬R) is satisfiable
    s.push()
    s.add(Not(R))

    print("Propositional Logic Syllogism with Z3:")
    
    print("--------------------------------------")
    if s.check() == unsat:
        print("The conclusion (R) follows from the premises.")
    else:
        print(
            "The conclusion does NOT necessarily follow from the premises in propositional logic."
        )
        print("Counterexample Model:")
        print(s.model())


def predicate_logic_syllogism_z3():
    from z3 import (
        Solver,
        ForAll,
        Implies,
        Function,
        BoolSort,
        Const,
        Not,
        And,
        unsat,
        prove,
        DeclareSort,
    )

    # Declare a sort for Person
    Person = DeclareSort("Person")

    # Define variables
    x = Const("x", Person)

    # Define predicates
    H = Function("H", Person, BoolSort())  # H(x): x is human
    M = Function("M", Person, BoolSort())  # M(x): x is mortal
    G = Function("G", Person, BoolSort())  # G(x): x is Greek

    # Major premise: ∀x (H(x) ⇒ M(x))
    major_premise = ForAll([x], Implies(H(x), M(x)))

    # Minor premise: ∀x (G(x) ⇒ H(x))
    minor_premise = ForAll([x], Implies(G(x), H(x)))

    # Conclusion: ∀x (G(x) ⇒ M(x))
    conclusion = ForAll([x], Implies(G(x), M(x)))

    # Attempt to prove that the conclusion follows from the premises
    print("Predicate Logic Syllogism with Z3:")
    print("------------------------------------")
    print("Attempting to prove that the conclusion follows from the premises...")

    # Use Z3's prove function
    prove(Implies(And(major_premise, minor_premise), conclusion))


# Run the functions
propositional_logic_syllogism_z3()
print("\n")
predicate_logic_syllogism_z3()

P & Q
Propositional Logic Syllogism with Z3:
--------------------------------------
The conclusion does NOT necessarily follow from the premises in propositional logic.
Counterexample Model:
[R = False, Q = True, P = True]


Predicate Logic Syllogism with Z3:
------------------------------------
Attempting to prove that the conclusion follows from the premises...
proved


In [ ]:
from z3 import *

# Create a Z3 solver
solver = Solver()

# Define the predicates
Greek = Function('Greek', BoolSort())
Human = Function('Human', BoolSort())
Mortal = Function('Mortal', BoolSort())
Socrates = Bool('Socrates')

# Define the statements
# Socrates is Greek
solver.add(Greek(Socrates))

# All humans are mortal
Humans = Function('Humans', BoolSort())
solver.add(ForAll([Humans], Implies(Humans, Mortal(Humans))))

# All Greeks are human
Greeks = Function('Greeks', BoolSort())
solver.add(ForAll([Greeks], Implies(Greeks, Human(Greeks))))

# Therefore, Socrates is mortal
solver.add(Implies(Greek(Socrates), Mortal(Socrates)))

# Check the satisfiability
result = solver.check()
result, solver.model() if result == sat else None

Z3Exception: b'index out of bounds'

## Test predicate functionality

In [11]:
statement = "H(x) >> M(x)"
from sympy import symbols, Function, Implies
from sympy.parsing.sympy_parser import parse_expr, standard_transformations

# Define the transformations to handle the parsing
transformations = (standard_transformations)

# Define the variables and functions
x = symbols('x')
H = Function('H')
M = Function('M')

# Define the string expression
p = "H(x) < M(x)"

# Parse the expression
parsed_expr = parse_expr(p, transformations=transformations, local_dict={'H': H, 'M': M})

# Print the parsed expression
print(parsed_expr.free_symbols)
sp.sympify(p)

{x}


H(x) < M(x)